# Setup

## Imports

In [ ]:
# Numpy, Scipy, Matplotlib
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Drake imports
import pydrake
from pydrake.all import (DiagramBuilder, RigidTransform, MathematicalProgram, RollPitchYaw,
                        RotationMatrix, Meshcat, MeshcatVisualizerParams, MeshcatVisualizerCpp,
                        InverseKinematics, Solve, SpatialInertia, UnitInertia, JacobianWrtVariable)

# Imports of other project files
import constants
import config

import plant.simulation
import plant.manipulator as manipulator

import ctrl.aux
import plant.pedestal

import visualization

# Other imports
import time

import quaternion

In [ ]:
# Matplotlib configuring
# USE FOR PAPER
# plt.style.use(['science', 'no-latex'])
# font = {'size'   : 14}
# matplotlib.rc('font', **font)
# default_figsize = (2*3,2*2)
# USE GENERALLY
font = {'size'   : 16}
matplotlib.rc('font', **font)
default_figsize = (16,8)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16,8)
matplotlib.rcParams['lines.linewidth'] = 5
matplotlib.rcParams["axes.xmargin"] = 0

## Drake initialization

In [ ]:
meshcat = Meshcat()
web_url = meshcat.web_url()

### Constants

In [ ]:
num_links = config.NumLinks.TWO
sys_consts = constants.nominal_sys_consts(num_links)
w_L = sys_consts.w_L
h_L = sys_consts.h_L
pedestal_x = plant.pedestal.PEDESTAL_Y_DIM

In [ ]:
# Pedestal
pedestal_x_dim = constants.PEDESTAL_X_DIM(num_links)

In [ ]:
contact_body_name = manipulator.data["contact_body_name"]

### Creat MBP

In [ ]:
builder = DiagramBuilder()

mbp_plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(
    builder, time_step=1e-3)
mbp_plant.set_stiction_tolerance(constants.v_stiction)
mbp_plant.set_penetration_allowance(0.001)

### Add bodies

In [ ]:
# Arm
manipulator.data["add_plant_function"](
    mbp_plant, sys_consts.m_M, sys_consts.r, sys_consts.mu, scene_graph = scene_graph
)

In [ ]:
pedestal_instance = plant.pedestal.AddPedestal(mbp_plant, num_links, weld_base=False)

In [ ]:
# Paper
paper_instance = mbp_plant.AddModelInstance("paper")
paper_dims = [
    constants.PLYWOOD_LENGTH,
    sys_consts.w_L,
    sys_consts.h_L
]
paper_body = mbp_plant.AddRigidBody(
    "paper_body0", paper_instance,
    SpatialInertia(1, p_PScm_E=np.array([0., 0., 0.]),
                   G_SP_E=UnitInertia.SolidBox(*paper_dims))
)

mbp_plant.RegisterCollisionGeometry(
    paper_body, RigidTransform(), pydrake.geometry.Box(*paper_dims), "paper_body0",
    pydrake.multibody.plant.CoulombFriction(1,1)
)
mbp_plant.RegisterVisualGeometry(
    paper_body, RigidTransform(), pydrake.geometry.Box(*paper_dims),
    "paper_body0", [0, 1, 0, 1])

mbp_plant.WeldFrames(
    mbp_plant.GetFrameByName(plant.pedestal.pedestal_base_name, pedestal_instance),
    mbp_plant.GetBodyByName("paper_body0").body_frame(),
    RigidTransform(
        RotationMatrix().MakeZRotation(-np.pi/2),
        [0, 0, constants.PLYWOOD_LENGTH+sys_consts.h_L/2+plant.pedestal.PEDESTAL_BASE_Z_DIM/2])
)

In [ ]:
# False body
empty_inertia = SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0))
false_body_instance = mbp_plant.AddModelInstance("false")
mbp_plant.AddRigidBody("false_body", false_body_instance, empty_inertia)

end_effector_z_rotation = mbp_plant.AddJoint(pydrake.multibody.tree.RevoluteJoint(
    "contact_body_name_z_rotation",
    mbp_plant.GetFrameByName(contact_body_name),
    mbp_plant.GetFrameByName("false_body"),
    [0, 0, 1],
    damping=0
))

In [ ]:
mbp_plant.Finalize()

### Build diagram

In [ ]:
meshcat_params = MeshcatVisualizerParams()
vis = MeshcatVisualizerCpp.AddToBuilder(
    builder,
    scene_graph.get_query_output_port(),
    meshcat,
    meshcat_params)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
ik_context = mbp_plant.GetMyContextFromRoot(context)
vis_context = vis.GetMyContextFromRoot(context)

# Create end effector traj

In [ ]:
opt_DT = 0.1
out_DT = 0.01

In [ ]:
# Relative to *pedestal* CoM
joint_position = [
    - (w_L - pedestal_x_dim/2),
    0,
    plant.pedestal.PEDESTAL_Z_DIM + sys_consts.h_L/2 - plant.pedestal.PEDESTAL_BASE_Z_DIM/2
]

In [ ]:
# Trajectory parameters
desired_contact_distance = (num_links.value - 2 + 0.5)*w_L
initial_z_speed = 0.02 # 2 cm per second
angular_speed = np.pi/30

desired_radius = np.sqrt(sys_consts.r**2 + desired_contact_distance**2)
offset_angle = np.arcsin(sys_consts.r/desired_radius)

In [ ]:
# Two legs: Leg 1 (straight) and leg 2 (circle)
## Leg 1 depends on where leg 2 starts, so need to do leg 2 first

## Leg 2
init_theta = -offset_angle
end_theta = np.pi-offset_angle
leg_2_duration = (end_theta - init_theta)/angular_speed
leg_2_ts = np.arange(0,leg_2_duration,opt_DT)

leg_2_thetas = np.interp(
   leg_2_ts,
    [0, leg_2_duration],
    [init_theta, end_theta],
)

leg_2_xs = joint_position[0] - np.cos(leg_2_thetas)*desired_radius
leg_2_ys = joint_position[1] * np.ones_like(leg_2_thetas)
leg_2_zs = joint_position[2] + np.sin(leg_2_thetas)*desired_radius# + sys_consts.r

## Leg 1
init_x = leg_2_xs[0]
init_y = joint_position[1]
init_z = joint_position[2] - 3*sys_consts.r
leg_1_end_z = leg_2_zs[0]

leg_1_duration = (leg_1_end_z - init_z)/initial_z_speed
leg_1_ts = np.arange(0,leg_1_duration,opt_DT)

leg_1_thetas = init_theta*np.ones_like(leg_1_ts)
leg_1_xs = init_x*np.ones_like(leg_1_ts)
leg_1_ys = init_y*np.ones_like(leg_1_ts)
leg_1_zs = np.interp(
   leg_1_ts,
    [0, leg_1_duration],
    [init_z, leg_1_end_z],
)

# Stich together
opt_ts = np.concatenate((leg_1_ts, leg_2_ts+leg_1_duration))
xs = np.concatenate((leg_1_xs, leg_2_xs))
ys = np.concatenate((leg_1_ys, leg_2_ys))
zs = np.concatenate((leg_1_zs, leg_2_zs))
thetas = np.concatenate((leg_1_thetas, leg_2_thetas))

In [ ]:
out_ts = np.arange(0, opt_ts[-1], out_DT)

## Check trajectory

In [ ]:
plt.figure()
plt.subplot(221)
plt.plot(opt_ts, xs)
plt.title("$x$")
plt.subplot(222)
plt.plot(opt_ts, ys)
plt.title("$y$")
plt.subplot(223)
plt.plot(opt_ts, zs)
plt.title("$z$")
plt.subplot(224)
plt.plot(opt_ts, thetas)
plt.title(r"$\theta$")
plt.show()

In [ ]:
plt.plot(xs, zs)

plt.gca().set_aspect("equal")

# Set margins

In [ ]:
# Margin from joint limit
q_margin = 20 * np.pi/180

# Get initial values

In [ ]:
# Grab bodies and frames
end_effector_body = mbp_plant.GetBodyByName(manipulator.data["contact_body_name"])
end_effector_frame = end_effector_body.body_frame()
false_body_frame = mbp_plant.GetBodyByName("false_body").body_frame()
pedestal_base_frame = mbp_plant.GetBodyByName(plant.pedestal.pedestal_base_name).body_frame()

In [ ]:
# Get initial position
starting_q = np.array([
    # Panda joints
    -1.90831538,
    -0.74013834,
    2.29209901,
    -2.58333082,
    4.52633766,
    0.60032527,
    -3.10678351,
    # Slack in z direction
    2,
    # Quaternion orientation of pedestal
    1,
    0,
    0,
    0,
    # Position of pedestal, taken as somewhat arbitrary initial values from a previous sucessful run
    -0.004,
    -0.6,
    plant.pedestal.PEDESTAL_BASE_Z_DIM/2
])
mbp_plant.SetPositions(ik_context, starting_q)
vis.Publish(vis_context)

In [ ]:
# Grab joint limits
lower_lims_raw = []
upper_lims_raw = []
vel_lims_raw = []
accel_lims_raw = []

for i in range(1,8):
    jnt = mbp_plant.GetJointByName("panda_joint" + str(i))
    lower_lims_raw.append(jnt.position_lower_limits()[0])
    upper_lims_raw.append(jnt.position_upper_limits()[0])
    
    lower_vel_limit = jnt.velocity_lower_limits()
    upper_vel_limit = jnt.velocity_upper_limits()
    assert -lower_vel_limit == upper_vel_limit
    vel_lims_raw.append(upper_vel_limit[0])

# I don't think the accel lims are in the URDF, but ther are here:
# https://frankaemika.github.io/docs/control_parameters.html?highlight=joint%20limit
accel_lims_raw = np.array([15, 7.5, 10, 12.5, 15, 20, 20])

lower_lims_raw = np.array(lower_lims_raw)
upper_lims_raw = np.array(upper_lims_raw)

vel_lims_raw = np.array(vel_lims_raw)

lower_lims = lower_lims_raw + q_margin
upper_lims = upper_lims_raw - q_margin
vel_lims = vel_lims_raw/2
accel_lims = accel_lims_raw

# Run optimization

## Set up geometries for collision checking

In [ ]:
# geometry_pairs = [
#     # ("paper_body0", contact_body_name),
#     ("paper_body0", "panda_hand"),
#     ("paper_body0", "panda_link8"),
#     ("paper_body0", "panda_link7"),
#     ("paper_body0", "panda_link6"),
#     ("paper_body0", "panda_link5"),
#     ("paper_body0", "panda_link4"),
#     ("paper_body0", "panda_link3"),
#     ("paper_body0", "panda_link2"),
#     ("paper_body0", "panda_link1"),
#     ("paper_body0", "panda_link0"),
#     ("pedestal_left_body", contact_body_name),
#     ("pedestal_left_body", "panda_hand"),
#     ("pedestal_left_body", "panda_link8"),
#     ("pedestal_left_body", "panda_link7"),
#     ("pedestal_left_body", "panda_link6"),
#     ("pedestal_left_body", "panda_link5"),
#     ("pedestal_left_body", "panda_link4"),
#     ("pedestal_left_body", "panda_link3"),
#     ("pedestal_left_body", "panda_link2"),
#     ("pedestal_left_body", "panda_link1"),
#     ("pedestal_left_body", "panda_link0"),
#     ("pedestal_right_body", contact_body_name),
#     ("pedestal_right_body", "panda_hand"),
#     ("pedestal_right_body", "panda_link8"),
#     ("pedestal_right_body", "panda_link7"),
#     ("pedestal_right_body", "panda_link6"),
#     ("pedestal_right_body", "panda_link5"),
#     ("pedestal_right_body", "panda_link4"),
#     ("pedestal_right_body", "panda_link3"),
#     ("pedestal_right_body", "panda_link2"),
#     ("pedestal_right_body", "panda_link1"),
#     ("pedestal_right_body", "panda_link0"),
#     ("pedestal_bottom_body", "panda_hand"),
#     ("pedestal_bottom_body", "panda_link8"),
#     ("pedestal_bottom_body", "panda_link7"),
#     ("pedestal_bottom_body", "panda_link6"),
#     ("pedestal_bottom_body", "panda_link5"),
#     ("pedestal_bottom_body", "panda_link4"),
#     ("pedestal_bottom_body", "panda_link3"),
#     ("pedestal_bottom_body", "panda_link2"),
#     ("pedestal_bottom_body", "panda_link1"),
#     ("pedestal_bottom_body", "panda_link0"),
# ]

# # TODO: get this from joints?
# adjacent_pairs = [
#     ("panda_link0", "panda_link1"),
#     ("panda_link1", "panda_link2"),
#     ("panda_link2", "panda_link3"),
#     ("panda_link3", "panda_link4"),
#     ("panda_link4", "panda_link5"),
#     ("panda_link5", "panda_link6"),
#     ("panda_link6", "panda_link7"),
#     ("panda_link6", "panda_link8"),
#     ("panda_link7", "panda_link8"),
#     ("panda_link7", "panda_hand"),
#     ("panda_link8", "panda_hand"),
# ]
# panda_links = ["panda_link" + str(i) for i in range(9)]
# panda_links += ["panda_hand"]
# for link_1 in panda_links:
#     for link_2 in panda_links:
#         if link_1 == link_2:
#             continue
#         if (link_1, link_2) in adjacent_pairs:
#             continue
#         if (link_2, link_1) in adjacent_pairs:
#             continue
#         if (link_2, link_1) in geometry_pairs:
#             continue
#         if (link_1, link_2) in geometry_pairs:
#             continue
#         geometry_pairs.append((link_1, link_2))
        
# geometry_pair_idxs = set()
# for name1, name2 in geometry_pairs:
#     idxs = [
#         int(mbp_plant.GetBodyByName(name1).index()),
#         int(mbp_plant.GetBodyByName(name2).index())
#     ]
#     geometry_pair_idxs.add(tuple(sorted(idxs)))

## Run IK

In [ ]:
# num_iter = 1000

In [ ]:
# pedestal_x_range = [-10, 10]
# pedestal_y_range = [-10, 10]
# pedestal_z_range = [constants.IN_TO_M*3/8, plant.pedestal.PEDESTAL_BASE_Z_DIM*2]
# def sample_q():
#     q_sample = []
#     rng = np.random.default_rng()
#     for lb, ub in zip(lower_lims, upper_lims):
#         angle_range = ub - lb
#         sample = rng.random()*angle_range + lb
#         q_sample.append(sample)
#     q_sample.append(rng.random()*np.pi*2 - np.pi)
    
#     q_sample += [1,0,0,0] # Pedestal orientation
    
#     # x
#     x_range = pedestal_x_range[1] - pedestal_x_range[0]
#     sample = rng.random()*x_range + pedestal_x_range[0]
#     q_sample.append(sample)
    
#     # y
#     y_range = pedestal_y_range[1] - pedestal_y_range[0]
#     sample = rng.random()*y_range + pedestal_y_range[0]
#     q_sample.append(sample)
    
#     # z
#     z_range = pedestal_z_range[1] - pedestal_z_range[0]
#     sample = rng.random()*z_range + pedestal_z_range[0]
#     q_sample.append(sample)
    
#     return q_sample

In [ ]:
# angle_tol = 1e-2
# pos_tol = 1e-3
# collision_buffer = 1e-3

In [ ]:
# def violates_singularity(qs):
#     last_theta_X = None
#     last_theta_Y = None
#     last_theta_Z = None
#     theta_diff = np.pi

#     last_qw = None
#     last_qx = None
#     last_qy = None
#     last_qz = None
#     q_diff = 0.5

#     first_iter = True

#     for i, q in enumerate(qs):
#         mbp_plant.SetPositions(ik_context, q)

#         ee_pose = mbp_plant.EvalBodyPoseInWorld(ik_context, end_effector_body)

#         rpy = RollPitchYaw(ee_pose.rotation()).vector()
#         quat_ = ee_pose.rotation().ToQuaternion()

#         if not first_iter:
#             if abs(rpy[0] - last_theta_X) > theta_diff:
#                 return True
#             if abs(rpy[1] - last_theta_Y) > theta_diff:
#                 return True
#             if abs(rpy[2] - last_theta_Z) > theta_diff:
#                 return True

#             if abs(quat_.w() - last_qw) > q_diff:
#                 return True
#             if abs(quat_.x() - last_qx) > q_diff:
#                 return True
#             if abs(quat_.y() - last_qy) > q_diff:
#                 return True
#             if abs(quat_.z() - last_qz) > q_diff:
#                 return True

#         last_theta_X = rpy[0]
#         last_theta_Y = rpy[1]
#         last_theta_Z = rpy[2]
#         last_qw = quat_.w()
#         last_qx = quat_.x()
#         last_qy = quat_.y()
#         last_qz = quat_.z()
#         first_iter = False
    
#     return False

In [ ]:
# def violates_fine_grain_collision(opt_qs):
#     for q in opt_qs:
#         mbp_plant.SetPositions(ik_context, q)
#         mbp_plant.get_actuation_input_port().FixValue(ik_context, [0]*7) # Have to do this to use output port
#         contact_results = mbp_plant.get_contact_results_output_port().Eval(ik_context)

#         for i in range(contact_results.num_point_pair_contacts()):
#             info = contact_results.point_pair_contact_info(i)
#             idx_pair = tuple(sorted([int(info.bodyA_index()), int(info.bodyB_index())]))
#             if idx_pair in geometry_pair_idxs:
#                 return True
#     return False

In [ ]:
# def violates_kinematic_singularity(opt_qs):
#     for q in opt_qs:
#         mbp_plant.SetPositions(ik_context, q)
#         mbp_plant.get_actuation_input_port().FixValue(ik_context, [0]*7) # Have to do this to use output port
#         contact_results = mbp_plant.get_contact_results_output_port().Eval(ik_context)

#         J = mbp_plant.CalcJacobianSpatialVelocity(
#             ik_context,
#             JacobianWrtVariable.kV,
#             mbp_plant.GetFrameByName(contact_body_name),
#             [0, 0, 0],
#             mbp_plant.world_frame(),
#             mbp_plant.world_frame())
#         min_svd = np.min(np.linalg.svd(J)[1])
#         print(min_svd)
#         if min_svd < 0.05:
#             return True
#     return False

In [ ]:
# opt_qs = []
# for _ in range(num_iter):
#     init_q = sample_q()
    
#     # Output arrays
#     opt_qs = []

#     is_success = True
#     pedestal_X = constants.IN_TO_M*22
#     pedestal_Y = -0.25
#     for t, theta, x, y, z in zip(opt_ts, thetas, xs, ys, zs):
#         # Create program
#         ik = InverseKinematics(mbp_plant, ik_context, with_joint_limits=False)

#         #Add collision constraints
#         for body_name_i, body_name_j in geometry_pairs:
#                 col_geos_i = mbp_plant.GetCollisionGeometriesForBody(mbp_plant.GetBodyByName(body_name_i))
#                 col_geos_j = mbp_plant.GetCollisionGeometriesForBody(mbp_plant.GetBodyByName(body_name_j))
                
#                 # For now, just use first element
#                 # This gives us a rougher check which runs faster; we check for finer grained collisions at the end
#                 col_geos_i = [col_geos_i[0]]
#                 col_geos_j = [col_geos_j[0]]
                
#                 for col_geo_i in col_geos_i:
#                     for col_geo_j in col_geos_j:
#                         geo_list = (col_geo_i, col_geo_j)
#                         ik.AddDistanceConstraint(
#                             distance_lower=collision_buffer, distance_upper=1e9, geometry_pair=geo_list)

#         # End effector constraints
#         ## Position
#         p_WG_lower = np.array([[x-pos_tol, y-pos_tol, z-pos_tol]]).T
#         p_WG_upper = np.array([[x+pos_tol, y+pos_tol, z+pos_tol]]).T
#         ik.AddPositionConstraint(
#             frameA=pedestal_base_frame,
#             frameB=end_effector_frame, p_BQ=np.zeros(3),
#             p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)
#         ## Orientation
#         ik.AddOrientationConstraint(
#             frameAbar=mbp_plant.world_frame(),
#             R_AbarA=RotationMatrix.MakeZRotation(np.pi/2),
#             frameBbar=false_body_frame,
#             R_BbarB=RotationMatrix.MakeXRotation(-theta-offset_angle),
#             theta_bound=angle_tol
#         )

#         # Pedestal constraints
#         ## Position
#         if len(opt_qs) == 0:
#             p_WG_lower = np.array([[pedestal_x_range[0], pedestal_y_range[0], pedestal_z_range[0]]]).T
#             p_WG_upper = np.array([[pedestal_x_range[1], pedestal_y_range[1], pedestal_z_range[1]]]).T
#         else:
#             p_WG_lower = np.array([[opt_qs[-1][-3], opt_qs[-1][-2], opt_qs[-1][-1]]]).T
#             p_WG_upper = np.array([[opt_qs[-1][-3], opt_qs[-1][-2], opt_qs[-1][-1]]]).T
#         ik.AddPositionConstraint(
#             frameA=mbp_plant.world_frame(),
#             frameB=pedestal_base_frame, p_BQ=np.zeros(3),
#             p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)
#         ## Orientation
#         ik.AddOrientationConstraint(
#                 frameAbar=mbp_plant.world_frame(), R_AbarA=RotationMatrix(),#.MakeZRotation(np.pi),
#                 frameBbar=pedestal_base_frame, R_BbarB=RotationMatrix(),
#                 theta_bound=1e-3
#             )
        
#         # Joint space constraints
#         ## Add joint limit constraints
#         for q_, lb, ub in zip(ik.q(), lower_lims, upper_lims):
#             ik.prog().AddConstraint(q_ >= lb)
#             ik.prog().AddConstraint(q_ <= ub)
#         if len(opt_qs) > 0:
#             # Add velocity constraints
#             for q_, vel_lim, prev_q_ in zip(ik.q(), vel_lims, opt_qs[-1]): # TODOL reorder
#                 ik.prog().AddConstraint(q_ >= prev_q_ - vel_lim*opt_DT)
#                 ik.prog().AddConstraint(q_ <= prev_q_ + vel_lim*opt_DT)
#                 vel = (q_ - prev_q_)/opt_DT
#                 ik.prog().AddCost(vel**2)
#         if len(opt_qs) > 1:
#             for prev_prev_q_, prev_q_, q_, accel_lim in zip(opt_qs[-2], opt_qs[-1], ik.q(), accel_lims):
#                 prev_vel = (prev_q_-prev_prev_q_)/opt_DT
#                 vel = (q_ - prev_q_)/opt_DT
#                 ik.prog().AddConstraint(vel >= prev_vel - accel_lim*opt_DT)
#                 ik.prog().AddConstraint(vel <= prev_vel + accel_lim*opt_DT)

#         # Seed
#         if len(opt_qs) == 0:
#             ik.prog().SetInitialGuess(ik.q(), init_q)
#         else:
#             ik.prog().SetInitialGuess(ik.q(), opt_qs[-1])

#         # Solve and grab values
#         result = Solve(ik.prog())
#         if not result.is_success():
#             is_success = False
#             print("==================")
#             print("Failed at time {} for iter {}".format(t, _))
#             print("Failed constraints:")
#             print("------------------")
#             print(result.GetInfeasibleConstraintNames(ik.prog()))
#             print("------------------")
#             print("==================")
#             break

#         opt_qs.append(result.GetSolution(ik.q()))
#     if is_success:
#         # TODO: refactor this to do the checks earlier
#         if violates_singularity(opt_qs):
#             is_success = False
#             print("==================")
#             print("Failed at time {} for iter {}".format(t, _))
#             print("Failed because of singularity violations.")
#             print("==================")
#             continue
#         if violates_fine_grain_collision(opt_qs):
#             is_success = False
#             print("==================")
#             print("Failed at time {} for iter {}".format(t, _))
#             print("Failed because of collisions.")
#             print("==================")
#             continue
#         if violates_kinematic_singularity(opt_qs):
#             is_success = False
#             print("==================")
#             print("Failed at time {} for iter {}".format(t, _))
#             print("Failed because of kinematic singularity.")
#             print("==================")
#             continue
#         print("Success!")
#         print("Initial position:", opt_qs[0])
#         break
# init_q = opt_qs[0]
# opt_qs = np.array(opt_qs)

## Interpolate

In [ ]:
# opt_qs = np.array(opt_qs)

In [ ]:
# out_qs = []
# for i in range(opt_qs.shape[1]):
#     out_qs.append(np.interp(
#        out_ts,
#         opt_ts,
#         opt_qs[:,i],
#     ))
# out_qs = np.array(out_qs).T

In [ ]:
out_qs_arm_ = np.load("qs_2_links.npz")["qs"]

In [ ]:
out_qs_pedestal_tran_ = np.load("pedestal_xyz_2_links.npz")["pedestal_xyz"]

In [ ]:
# Add pedestal orientation and z slack, which sbouldn't matter now
out_qs_2_no_repeats = np.concatenate(([0, 1,0,0,0], out_qs_pedestal_tran_))

In [ ]:
out_qs_2_ = np.repeat(np.expand_dims(out_qs_2_no_repeats,0), repeats=out_qs_arm_.shape[0], axis=0)

In [ ]:
out_qs = np.hstack((out_qs_arm_, out_qs_2_))

In [ ]:
out_qs.shape

# Results

## Visualize in meshcat

In [ ]:
assert False

In [ ]:
t = 0
dt = out_DT
xs_out = []
ys_out = []
zs_out = []
theta_Xs_out = []
theta_Ys_out = []
theta_Zs_out = []
theta_Ls_out = []
theta_L_EE_out = []

q_ws = []
q_xs = []
q_ys = []
q_zs = []

for q in out_qs:
    mbp_plant.SetPositions(ik_context, q)

    # Evaluate end effector pose
    X_W_EE = mbp_plant.EvalBodyPoseInWorld(ik_context, end_effector_body)
    xs_out.append(X_W_EE.translation()[0])
    ys_out.append(X_W_EE.translation()[1])
    zs_out.append(X_W_EE.translation()[2])
    theta_Xs_out.append(RollPitchYaw(X_W_EE.rotation()).vector()[0])
    theta_Ys_out.append(RollPitchYaw(X_W_EE.rotation()).vector()[1])
    theta_Zs_out.append(RollPitchYaw(X_W_EE.rotation()).vector()[2])
    
    q_ws.append(X_W_EE.rotation().ToQuaternion().w())
    q_xs.append(X_W_EE.rotation().ToQuaternion().x())
    q_ys.append(X_W_EE.rotation().ToQuaternion().y())
    q_zs.append(X_W_EE.rotation().ToQuaternion().z())

    # Visualize
    visualization.AddMeshcatTriad(meshcat, "man_pose", X_PT=X_W_EE)
    false_pose = mbp_plant.EvalBodyPoseInWorld(
        ik_context, mbp_plant.GetBodyByName("false_body"))
    visualization.AddMeshcatTriad(meshcat, "false_pose", X_PT=false_pose, opacity=0.3)
    
    pedestal_xyz = np.array(q[-3:])+np.array(joint_position)
    theta_L = RollPitchYaw(false_pose.rotation()).vector()[0]
    if t > leg_1_duration: # Only update orientation map during leg2
        theta_Ls_out.append(theta_L)
    
    X_W_L = RigidTransform(p=[
        pedestal_xyz[0] - desired_contact_distance*np.cos(theta_L),
        pedestal_xyz[1],
        pedestal_xyz[2] + desired_contact_distance*np.sin(theta_L)
    ], R=RotationMatrix.MakeYRotation(theta_L))
    visualization.AddMeshcatTriad(meshcat, "link_pose", X_PT=X_W_L)

    X_L_EE = X_W_L.inverse().multiply(X_W_EE)
    R_L_EE = X_L_EE.rotation()
    theta_L_EE_Z = RollPitchYaw(R_L_EE).vector()[2]
    target_pose = X_W_L.multiply(RigidTransform(R=RotationMatrix.MakeZRotation(theta_L_EE_Z)))
    visualization.AddMeshcatTriad(meshcat, "target_pose", X_PT=target_pose)
    if t > leg_1_duration: # Only update orientation map during leg2
        theta_L_EE_out.append(theta_L_EE_Z)

    # Publish and increment time
    vis.Publish(vis_context)
    time.sleep(dt)
    t += dt

xs_out = np.array(xs_out)
ys_out = np.array(ys_out)
zs_out = np.array(zs_out)
theta_Xs_out = np.array(theta_Xs_out)
theta_Ys_out = np.array(theta_Ys_out)
theta_Zs_out = np.array(theta_Zs_out)
theta_Ls_out = np.array(theta_Ls_out)
theta_L_EE_out = np.array(theta_L_EE_out)

## Plot results

In [ ]:
plt.plot(out_ts, ys_out)
plt.show()

In [ ]:
plt.figure()
# TODO: Subtract off pedestal values instead of initial values
plt.plot(xs-xs[0]+xs_out[0], zs-zs[0]+zs_out[0], label="Desired", color='purple', linestyle='--')
plt.plot(xs_out, zs_out, label="Actual", color='purple', alpha=0.3)
plt.gca().set_aspect("equal")

In [ ]:
plt.figure()
plt.plot(out_ts, theta_Xs_out, label=r"$\theta_X$", color='red')
plt.plot(out_ts, theta_Ys_out, label=r"$\theta_Y$", color='green')
plt.plot(out_ts, theta_Zs_out, label=r"$\theta_Z$", color='blue')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(out_ts, q_ws, label=r"$w$", color='gray', linewidth=3, marker='o')
plt.plot(out_ts, q_xs, label=r"$x$", color='r', linewidth=3, marker='o')
plt.plot(out_ts, q_ys, label=r"$y$", color='g', linewidth=3, marker='o')
plt.plot(out_ts, q_zs, label=r"$z$", color='b', linewidth=3, marker='o')
plt.legend()
plt.show()

In [ ]:
for i in range(8):
    plt.subplot(2, 4, i+1)

    plt.plot(out_ts, np.array(out_qs)[:,i])
    if i < 7:
        plt.ylim([-np.pi*1.1, np.pi*1.1])
    plt.ylabel("Angle (radians)")
    plt.xlabel("Time (seconds)")
    plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(theta_Ls_out, theta_L_EE_out)
plt.show()

## Query meshcat visualization

In [ ]:
query_idx = 2000

In [ ]:
## Select what position to visualize
# mbp_plant.SetPositions(ik_context, out_qs[0])
mbp_plant.SetPositions(ik_context, out_qs[query_idx])
# mbp_plant.SetPositions(ik_context, starting_q)
# mbp_plant.SetPositions(ik_context, plant.manipulator.neutral_q + [0])

ee_pose = mbp_plant.EvalBodyPoseInWorld(ik_context, end_effector_body)
visualization.AddMeshcatTriad(meshcat, "man_pose", X_PT=ee_pose)
false_pose = mbp_plant.EvalBodyPoseInWorld(
    ik_context, mbp_plant.GetBodyByName("false_body"))
visualization.AddMeshcatTriad(meshcat, "false_pose", X_PT=false_pose, opacity=0.3)
vis.Publish(vis_context)

# Save results

In [ ]:
# This prevents the whole script from running through when I do "run all cells",
# which I want to do without necessarily overriding existing outputs
assert False

In [ ]:
file_qs = np.array(out_qs)[:,:7]

In [ ]:
# np.savez("qs_{}_links.npz".format(num_links.value), qs=file_qs)

In [ ]:
# poses = []
# for (x_, y_, z_, q_w_, q_x_, q_y_, q_z_) in zip(xs_out, ys_out, zs_out, q_ws, q_xs, q_ys, q_zs):
#     pos_dict = {}
#     pos_dict["position"]  = [x_, y_, z_]
#     pos_dict["orientation"] = quaternion.quaternion(q_w_, q_x_, q_y_, q_z_)
#     poses.append(pos_dict)
# np.savez("x0s_{}_links.npz".format(num_links.value), poses=poses)

In [ ]:
# np.savez("starting_q_{}_links.npz".format(num_links.value), starting_q=file_qs[0])

In [ ]:
# np.savez("pedestal_xyz_{}_links.npz".format(num_links.value), pedestal_xyz=out_qs[0][-3:])

In [ ]:
np.savez("x0s_sim_{}_links.npz".format(num_links.value), poses=np.vstack([
        theta_Xs_out,
        theta_Ys_out,
        theta_Zs_out,
        xs_out,
        ys_out,
        zs_out
    ]), ts=out_ts[:-1],
)

In [ ]:
np.savez("orientation_map_{}_links.npz".format(num_links.value), 
    theta_Ls=theta_Ls_out,
    theta_L_EE=theta_L_EE_out
)

# Tests

In [ ]:
test2_ = np.load("x0s_sim_2_links.npz")
test4_ = np.load("x0s_sim_4_links.npz")

In [ ]:
test2_["poses"].shape
test2_["ts"].shape

In [ ]:
test4_["poses"].shape

# Backwards

In [ ]:
import warnings
from pydrake.all import Quaternion, RollPitchYaw


In [ ]:
def process_poses(poses):
    xs = []
    ys = []
    zs = []
    theta_xs = []
    theta_ys = []
    theta_zs = []
    q_ws = []
    q_xs = []
    q_ys = []
    q_zs = []
    
    for pose in poses:
        xs.append(pose["position"][0])
        ys.append(pose["position"][1])
        zs.append(pose["position"][2])

        quat_vec = np.array([
            pose['orientation'].w,
            pose['orientation'].x,
            pose['orientation'].y,
            pose['orientation'].z
        ])

        q_ws.append(quat_vec[0])
        q_xs.append(quat_vec[1])
        q_ys.append(quat_vec[2])
        q_zs.append(quat_vec[3])
        
        quat = Quaternion(
            w=pose['orientation'].w,
            x=pose['orientation'].x,
            y=pose['orientation'].y,
            z=pose['orientation'].z,
        )
        rpy = RollPitchYaw(quat).vector()
        theta_xs.append(rpy[0])
        theta_ys.append(rpy[1])
        theta_zs.append(rpy[2])
    return {
        'x': np.array(xs),
        'y': np.array(ys),
        'z': np.array(zs),
        'theta_x': np.array(theta_xs),
        'theta_y': np.array(theta_ys),
        'theta_z': np.array(theta_zs),
        'q_w': np.array(q_ws), 
        'q_x': np.array(q_xs),
        'q_y': np.array(q_ys),
        'q_z': np.array(q_zs),
    }

In [ ]:
def process_poses_log(poses):
    xs = poses[()]['x']
    ys = poses[()]['y']
    zs = poses[()]['z']
    theta_xs = []
    theta_ys = []
    theta_zs = []
    q_ws = []
    q_xs = []
    q_ys = []
    q_zs = []
    
    for i in range(len(poses[()]['x'])):
        quat = Quaternion(
            w=poses[()]['q_w'][i],
            x=poses[()]['q_x'][i],
            y=poses[()]['q_y'][i],
            z=poses[()]['q_z'][i],
        )
        rpy = RollPitchYaw(quat).vector()
        theta_xs.append(rpy[0])
        theta_ys.append(rpy[1])
        theta_zs.append(rpy[2])

        q_ws.append(poses[()]['q_w'][i])
        q_xs.append(poses[()]['q_x'][i])
        q_ys.append(poses[()]['q_y'][i])
        q_zs.append(poses[()]['q_z'][i])
    return {
        'x': xs,
        'y': ys,
        'z': zs,
        'theta_x': np.array(theta_xs),
        'theta_y': np.array(theta_ys),
        'theta_z': np.array(theta_zs),
        'q_w': np.array(q_ws), 
        'q_x': np.array(q_xs),
        'q_y': np.array(q_ys),
        'q_z': np.array(q_zs),
    }

In [ ]:
def load_data(run_num, gain, prefix="q_ee"):
    pref_path = "/Users/dani/Documents/lis/notes/testing/2022-04-29/"
    # Load npz
    log_npz = np.load(pref_path + "test1/runs/run{}/{}_log.npz".format(run_num, prefix), allow_pickle=True)
    
    # Handle time
    dt = log_npz["dt"][()]
    end_time = log_npz["end_time"][()]
    act_t = log_npz['t_log'] - log_npz['t_log'][0]
    des_t = np.arange(0, end_time, dt)
    
    print("dt:", dt)
    print("end_time:", end_time)

    # Pull out trajectories
    poses_fixed__npz = np.load(pref_path + "test1/x0s/x0s.npz".format(end_time, dt), allow_pickle=True)
    position_traj__npz = np.load(pref_path + "test1/qs/qs.npz".format(end_time, dt), allow_pickle=True)
    
    # Suppress numba warnings and prcess dicts
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        dict_des_ee_poses = poses_fixed__npz["poses"]
        if 'ee_log' in log_npz.files:
            log_act_ee_poses = log_npz['ee_log']

        des_qs = position_traj__npz['qs']
        if 'q_log' in log_npz.files:
            dict_act_qs = log_npz['q_log']

    if 'q_log' in log_npz.files:
        act_qs = process_q_dict(dict_act_qs)
    else:
        act_qs = np.zeros((len(act_t), 7))*np.nan
    if 'ee_log' in log_npz.files:
        act_ee_poses = process_poses_log(log_act_ee_poses)
    else:
        act_ee_poses = {
            'x': np.zeros_like(act_t)*np.nan,
            'y': np.zeros_like(act_t)*np.nan,
            'z': np.zeros_like(act_t)*np.nan,
            'theta_x': np.zeros_like(act_t)*np.nan,
            'theta_y': np.zeros_like(act_t)*np.nan,
            'theta_z': np.zeros_like(act_t)*np.nan
        }

    des_ee_poses = process_poses(dict_des_ee_poses)

    out_dict = {
        "q": {
            "act": act_qs,
            "des": des_qs,
        },
        "ee_pose": {
            "act": act_ee_poses,
            "des": des_ee_poses
        },
        "t": {
            "act": act_t,
            "des": des_t
        },
        "dt": dt,
        "end_time": end_time,
        "gain": gain
    }
    return out_dict

In [ ]:
run_data = load_data(1, 40, prefix="data")

In [ ]:
run_data

In [ ]:
joint_position = 

In [ ]:
plt.figure()
labels = ["Desired", "Actual"]
keys = ["des", "act"]
cmaps = ["cool", "hot"]

for label, key, cmap in zip(labels, keys, cmaps):
    plt.scatter(
        run_data['ee_pose'][key]['x'],
        run_data['ee_pose'][key]['z'],
        c=run_data['t'][key],
        label=label, cmap=cmap)
    

plt.scatter([0.48199032], [0.53412184])
plt.xlabel("$x$ position (m)")
plt.ylabel("$z$ position (m)")
plt.gca().set_aspect("equal")
plt.legend()
plt.show()

In [ ]:
prog = MathematicalProgram()

x_0 = prog.NewContinuousVariables(1, 1, name="x_0")
z_0 = prog.NewContinuousVariables(1, 1, name="y_0")
r = prog.NewContinuousVariables(1, 1, name="r")

for x, z in zip(run_data['ee_pose']["des"]['x'], run_data['ee_pose']["des"]['z']):
    lhs = (x-x_0[0,0])**2 + (z-z_0[0,0])**2
    rhs = r[0,0]**2
    prog.AddCost((lhs - rhs)**2)
    prog.AddConstraint(r[0,0] >= 0.001)
result = Solve(prog)
if result.is_success():
    print(result.GetSolution())

In [ ]:
plt.figure()
plt.plot(run_data['ee_pose']["des"]['x'], run_data['ee_pose']["des"]['z'])
plt.scatter([result.GetSolution()[0]], [result.GetSolution()[1]], color='k')
angs = np.linspace(0, np.pi*2, 200)
plt.plot(result.GetSolution()[2]*np.cos(angs)+result.GetSolution()[0],
         result.GetSolution()[2]*np.sin(angs)+result.GetSolution()[1],
         linestyle='--', color='k')
plt.gca().set_aspect("equal")

In [ ]:
p_WJ = np.array([result.GetSolution()[0], run_data['ee_pose']['des']['y'][0], result.GetSolution()[1]])
p_PJ = np.array(joint_position)+np.array([0,0,sys_consts.r])

In [ ]:
link_base = p_WJ - np.array([0,0,sys_consts.r])

In [ ]:
t = 0
theta_Ls_out = []
theta_L_EE_out = []
ts = []
for q in run_data['q']['des']:
    q_ = np.concatenate((q, [0, 1, 0, 0, 0], p_WP))
    mbp_plant.SetPositions(ik_context, q_)

    # Evaluate end effector pose
    X_W_EE = mbp_plant.EvalBodyPoseInWorld(ik_context, end_effector_body)

    # Visualize
    visualization.AddMeshcatTriad(meshcat, "man_pose", X_PT=X_W_EE)
    false_pose = mbp_plant.EvalBodyPoseInWorld(
        ik_context, mbp_plant.GetBodyByName("false_body"))
    visualization.AddMeshcatTriad(meshcat, "false_pose", X_PT=false_pose, opacity=0.3)

    X_W_J = RigidTransform(p=link_base)
    visualization.AddMeshcatTriad(meshcat, "X_W_J", X_PT=X_W_J)
    
    p_W_EE = X_W_EE.translation()
    p_J_EE = p_W_EE - link_base
    angle = np.arctan2(p_J_EE[2], p_J_EE[0])
    # man_angle = 
    theta_L = angle - offset_angle #RollPitchYaw(false_pose.rotation()).vector()[0]
    
    theta_Ls_out.append(theta_L)
    X_J_L = RigidTransform(
        p=[np.cos(theta_L)*sys_consts.w_L/2, 0, np.sin(theta_L)*sys_consts.w_L/2],
        R=RotationMatrix.MakeYRotation(-theta_L)
    )
    X_W_L = X_W_J.multiply(X_J_L)
    
    # X_W_L = RigidTransform(p=[
    #     pedestal_xyz[0] - desired_contact_distance*np.cos(theta_L),
    #     pedestal_xyz[1],
    #     pedestal_xyz[2] + desired_contact_distance*np.sin(theta_L)
    # ], R=RotationMatrix.MakeYRotation(theta_L))
    # visualization.AddMeshcatTriad(meshcat, "joint", X_PT=X_W_L)
    visualization.AddMeshcatTriad(meshcat, "X_W_L", X_PT=X_W_L)

    X_L_EE = X_W_L.inverse().multiply(X_W_EE)
    R_L_EE = X_L_EE.rotation()
    theta_L_EE_Z = RollPitchYaw(R_L_EE).vector()[2]
    target_pose = X_W_L.multiply(RigidTransform(R=RotationMatrix.MakeZRotation(theta_L_EE_Z)))
    visualization.AddMeshcatTriad(meshcat, "target_pose", X_PT=target_pose)
    theta_L_EE_out.append(theta_L_EE_Z)

    # Publish and increment time
    vis.Publish(vis_context)
    ts.append(t)
    time.sleep(dt)
    t += dt

# xs_out = np.array(xs_out)
# ys_out = np.array(ys_out)
# zs_out = np.array(zs_out)
# theta_Xs_out = np.array(theta_Xs_out)
# theta_Ys_out = np.array(theta_Ys_out)
# theta_Zs_out = np.array(theta_Zs_out)
ts = np.array(ts)
theta_Ls_out = np.array(theta_Ls_out)
theta_L_EE_out = np.array(theta_L_EE_out)

In [ ]:
ts

In [ ]:
plt.figure()
plt.plot(theta_Ls_out, theta_L_EE_out)
plt.show()

In [ ]:
np.savez("pedestal_xyz_{}_links.npz".format(num_links.value), pedestal_xyz=p_WP)

In [ ]:
np.savez("orientation_map_{}_links.npz".format(num_links.value), 
    theta_Ls=theta_Ls_out,
    theta_L_EE=theta_L_EE_out
)

In [ ]:
np.savez("x0s_sim_{}_links.npz".format(num_links.value), poses=np.vstack([
        run_data['ee_pose']['des']['theta_x'],
        run_data['ee_pose']['des']['theta_y'],
        run_data['ee_pose']['des']['theta_z'],
        run_data['ee_pose']['des']['x'],
        run_data['ee_pose']['des']['y'],
        run_data['ee_pose']['des']['z']
    ]), ts=ts,
)

In [ ]:
run_data['q']['des'][0]

In [ ]:
np.savez("starting_q_{}_links.npz".format(num_links.value), starting_q=run_data['q']['des'][0])